In [2]:
from scipy.linalg import lu
import numpy as np
A = np.loadtxt('/home/taosimin/iEDA24/iEDA/bin/matrix.txt')
print(A.shape)
print(A[0])

(29341, 29341)
[ 162.341286 -162.341286    0.       ...    0.          0.
    0.      ]


In [3]:
current = np.loadtxt('/home/taosimin/iEDA24/iEDA/bin/current.txt')
print(current.shape)
print(current)

(29341,)
[ 0.000000e+00  0.000000e+00  0.000000e+00 ... -2.399728e-10 -2.399728e-10
  8.100000e+05]


In [4]:
voltage = np.loadtxt('/home/taosimin/iEDA24/iEDA/bin/voltage.txt')
print(voltage.shape)
print(voltage)

(29341,)
[ 8.076774e-01  8.076774e-01  8.076906e-01 ... -2.998906e+05 -2.998906e+05
  8.100000e-01]


In [13]:
init = np.ones_like(voltage) * 0.78

In [15]:
import numpy as np

def feature_enhancer(feature, target, init=None, tol=1e-6, maxiter=100):
    N = len(target)
    if init is None:
        enhanced_feature = np.zeros(N, dtype=float)
    else:
        enhanced_feature = np.array(init, dtype=float)
        if not (enhanced_feature.shape == (N,1) or enhanced_feature.shape == (N,)):
            raise ValueError('shapes of feature {} and x0 {} are incompatible'
               .format(feature.shape, enhanced_feature.shape))
    enhanced_feature = enhanced_feature.ravel()

    residual = target - feature.dot(enhanced_feature)
    p = residual.copy()
    r_dot_r = np.dot(residual, residual)

    for i in range(maxiter):
        Ap = feature.dot(p)
        alpha = r_dot_r / np.dot(p, Ap)
        enhanced_feature += alpha * p
        residual -= alpha * Ap

        r_dot_r_new = np.dot(residual, residual)
        beta = r_dot_r_new / r_dot_r
        p = residual + beta * p
        r_dot_r = r_dot_r_new

        if r_dot_r < tol:
            break
    return enhanced_feature, i


In [16]:
enhanced_feature, num_iter =feature_enhancer(A, current, init=init, tol=1e-6, maxiter=100)
print("enhanced_feature: ", enhanced_feature)
print("num_iter: ", num_iter)

enhanced_feature:  [0.78       0.77999999 0.77999999 ... 0.78000001 0.78       0.80999971]
num_iter:  99


In [17]:
print(init)
print(init.shape)

[0.78 0.78 0.78 ... 0.78 0.78 0.78]
(29341,)


In [18]:
import numpy as np
from scipy.sparse.linalg import cg

# Solve the system AX = b using the Conjugate Gradient method
x, info = cg(A, current, x0=init, tol=1e-6, maxiter=10000)

# Print the solution and the convergence information
print("Solution X:\n", x)
print("Convergence info:", info)

Solution X:
 [0.78      0.78      0.78      ... 0.78      0.78      0.8099997]
Convergence info: 0
